In [50]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set the default template to 'plotly_dark' for all Plotly figures
import plotly.io as pio
pio.templates.default = 'plotly_dark'

In [51]:
df = pd.read_csv('../data/clean/clustered.csv').set_index('id')
df.head()

,age,gender,income,spending_score,membership_years,purchase_frequency,preferred_category,last_purchase_amount,age_range,income_level,spending_score_category,purchase_frequency_category,membership_score,cluster
id,,,,,,,,,,,,,,
1,38,Female,99342,90,3,24,Groceries,113.53,"(30, 40]",Medium,Premium,Medium,Bronze,1
2,21,Female,78852,60,2,42,Sports,41.93,"(20, 30]",Medium,High,High,Bronze,1
3,60,Female,126573,30,2,28,Clothing,424.36,"(50, 60]",High,Medium,Medium,Bronze,1
4,40,Other,47099,74,9,5,Home & Garden,991.93,"(30, 40]",Low,High,Low,Gold,2
5,65,Female,140621,21,3,25,Electronics,347.08,"(60, 70]",High,Low,Medium,Bronze,1


In [52]:
df['cluster'] = df['cluster'].astype('object')

In [53]:
df.select_dtypes('object').columns

Index(['gender', 'preferred_category', 'age_range', 'income_level',
       'spending_score_category', 'purchase_frequency_category',
       'membership_score', 'cluster'],
      dtype='object')

In [55]:
dimensions = df.select_dtypes('object').columns

for dimension in dimensions:

    data = df.groupby(['cluster', dimension]).size().reset_index(name='count')
    data['group_total'] = data.groupby('cluster')['count'].transform('sum')
    data['pct_of_group_total'] = (data['count'] / data['group_total']) * 100

    x = data['cluster']
    y = data['pct_of_group_total']
    fig = px.bar(data, x=x, y=y, color=dimension)
    fig.show()

ValueError: cannot insert cluster, already exists

In [ ]:
data = df.groupby('cluster')[['last_purchase_amount', 'purchase_frequency']].sum()

for col in data.columns:
    x = data.index
    y = data[col]
    fig = px.bar(data, x=x, y=y)
    fig.show()

/var/folders/v0/pfwld_t92_78n8y3m0zgsw2w0000gn/T/ipykernel_56436/69767371.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
data = df.groupby('cluster')[['last_purchase_amount', 'purchase_frequency']].mean()

for col in data.columns:
    x = data.index
    y = data[col]
    fig = px.bar(data, x=x, y=y)
    fig.show()

/var/folders/v0/pfwld_t92_78n8y3m0zgsw2w0000gn/T/ipykernel_56436/286203143.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
df.columns

Index(['age', 'gender', 'income', 'spending_score', 'membership_years',
       'purchase_frequency', 'preferred_category', 'last_purchase_amount',
       'age_range', 'income_level', 'spending_score_category',
       'purchase_frequency_category', 'membership_score', 'cluster'],
      dtype='object')

In [ ]:
fig = px.scatter(df, x=df.purchase_frequency, y=df.last_purchase_amount, color='cluster', size='spending_score')
fig.show()

In [ ]:
data

,last_purchase_amount,purchase_frequency
cluster,,
0,478.818747,36.556845
1,333.830094,19.481132
2,716.413625,18.505976
